<a href="https://colab.research.google.com/github/veroorli/ProjetProg/blob/master/TP9_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BDLE 2022

date du document  :  8/12/2022

# TP 9 : traitement sur des partitions : tri, regroupement, fenetre, jointure
Version LINA MEZDOUR



## Préparation

### Dataframe les Films

In [ ]:
# schema_film = "nf long, titre String, g Array<String>"
schema_film = StructType([StructField('nF',LongType()),
                          StructField('titre',StringType()),
                          StructField('g',ArrayType(StringType()))])

films = spark.read.json(local_dir + "/" + "films.json", schema = schema_film).selectExpr("nF", "titre", "g as genres")
# print('schema:', films.schema)

# extrait
film_extrait = films.where("nF <100").repartition(3).persist()

film_extrait.printSchema()
display(film_extrait)

root
 |-- nF: long (nullable = true)
 |-- titre: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)



,nF,titre,genres
0,94,Beautiful Girls (1996),"[Comedy, Drama, Romance]"
1,45,To Die For (1995),"[Comedy, Drama, Thriller]"
2,58,"Postman, The (Postino, Il) (1994)","[Comedy, Drama, Romance]"
3,54,"Big Green, The (1995)","[Children, Comedy]"
4,9,Sudden Death (1995),[Action]
...,...,...,...
87,38,It Takes Two (1995),"[Children, Comedy]"
88,15,Cutthroat Island (1995),"[Action, Adventure, Romance]"
89,85,Angels and Insects (1995),"[Drama, Romance]"
90,64,Two if by Sea (1996),"[Comedy, Romance]"


### Dataframe Les Notes

In [ ]:
notes_schema = StructType([StructField('nF',LongType()),
                           StructField('nU',LongType()),
                           StructField('note',DoubleType()),
                           StructField('annee',LongType())])

notes = spark.read.json(local_dir + "/" + "notes1M.json", schema = notes_schema).selectExpr("nF", "nU", "note", "annee")

#extrait
notes_extrait = notes.where("nU < 1000").join(film_extrait, "nF").select(notes["nF"], "nU", "note", "annee").repartition(3).persist()
print(notes_extrait.count())
display(notes_extrait)

185


,nF,nU,note,annee
0,19,76,1.5,2011
1,62,691,5.0,1996
2,25,832,5.0,1998
3,5,296,3.5,2005
4,48,781,3.0,1996
...,...,...,...,...
95,69,798,3.0,2000
96,95,430,3.0,1996
97,10,984,4.0,1996
98,36,261,2.0,1997


# Exercice 1 : Traitement sur des partitions

#### Fonction showPartitionSize

La fonction *showPartitionSize*  affiche le nombre d'éléments dans chaque partition

In [ ]:
# La fonction part_size prend en paramètres un numéro de partition (partID) 
#et un itérateur sur une partition.
# Elle retourne un itérateur sur une partition qui contient un seul tuple 
(num_partition, nbtuples).

def partSize(partID, iterateur):
  size=0
  suivant = next(iterateur, None)
  while suivant is not None :
    size += 1
    suivant = next(iterateur, None)
  yield (partID, size)


def showPartitionSize(df):
  #invoquer la fonction part_size sur chaque partition 
  t = df.selectExpr("1").rdd.mapPartitionsWithIndex(partSize)
  .toDF(['partID', 'size'])

  # Rmq : selectExpr("1") sert à simplifie la partition pour ne 
  #garder une seule colonne contentant des "1",
  # ce qui est suffisant pour compter le nombre de tuples dans la partition.

  #affichage
  t.show()
  return t

print('showPartitionSize définie')

showPartitionSize définie


#### Fonction showPartitions
 
La fonction *showPartitions*  affiche les _n_ premiers éléments de chaque partition


In [ ]:
def showPartitions(df, N=5):
  nb_partitions = df.rdd.getNumPartitions()

  # fonction topN est invoquée sur une partition.
  # topN retourne un itérateur sur une partition qui contient 
  #les N premiers éléments de cette partition
  def topN(partID, iterateur):
    c=0
    suivant = next(iterateur, None)
    while suivant is not None and c < N :
      c+=1
      suivant = next(iterateur, None)
      tuple_avec_numero_partition = (partID, *suivant)
      yield tuple_avec_numero_partition
    if c == 0:
      return []
  #-- fin de topN ---

  nom_attributs = ['partID'] + df.schema.fieldNames()
  premiers_tuples = df.rdd.mapPartitionsWithIndex(topN).toDF(nom_attributs)
  # afficher premiers_tuples qui contient le début de chaque partition 
  premiers_tuples.show(N*nb_partitions, False)

  # afficher séparément le contenu de chaque partition
  # for partID in range(nb_partitions):
  #   print("partition", partID)
  #   # afficher la partition partID
  #   premiers_tuples.where(f"partID={partID}").show(N, False)
    
print('showPartitions définie')

showPartitions définie


Afficher la taille de chaque partition

In [ ]:
showPartitionSize(film_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  31|
|     1|  31|
|     2|  30|
+------+----+



DataFrame[partID: bigint, size: bigint]

Afficher les premiers éléments de chaque partition

In [ ]:
showPartitions(film_extrait,5)

+------+---+-------------------------------------------------------------------------------+-----------------------------+
|partID|nF |titre                                                                          |genres                       |
+------+---+-------------------------------------------------------------------------------+-----------------------------+
|0     |45 |To Die For (1995)                                                              |[Comedy, Drama, Thriller]    |
|0     |58 |Postman, The (Postino, Il) (1994)                                              |[Comedy, Drama, Romance]     |
|0     |54 |Big Green, The (1995)                                                          |[Children, Comedy]           |
|0     |9  |Sudden Death (1995)                                                            |[Action]                     |
|0     |4  |Waiting to Exhale (1995)                                                       |[Comedy, Drama, Romance]     |
|1     |71 |Fair

## Question 1 : Tri des films par titre

En vous inspirant de la fonction *topN*  définir une fonction qui tri les données de *film_extrait* par titre. Le résultat a le meme schéma que film.

In [ ]:
def tri_par_titre(df, k = 4):
  nb_partitions = df.rdd.getNumPartitions()

  def hashFun(title):
    return ord(title[0].upper())-ord('A')


  def addHashNumber(partID, iterateur):
    suivant = next(iterateur, None)
    while suivant is not None :
      yield (*suivant, hashFun(suivant["titre"]))
      suivant = next(iterateur, None)

  nom_attributs = df.schema.fieldNames() + ['HushNumber']
  
  # hashed_df = df.rdd.mapPartitionsWithIndex(addHashNumber)
  #.toDF(nom_attributs).repartition(26, col("HushNumber")).persist()
  hashed_df = df.rdd.mapPartitionsWithIndex(addHashNumber)
  .toDF(nom_attributs).repartitionByRange(k, col("HushNumber"))
  .persist()


  showPartitionSize(hashed_df)

  hashed_df = hashed_df.withColumn("partID", spark_partition_id())
  .sortWithinPartitions(col("titre"))

  nb_partitions = hashed_df.rdd.getNumPartitions()

  # afficher séparément le contenu de chaque partition
  for partID in range(nb_partitions):
    print("partition", partID)
    # afficher la partition partID
    hashed_df.where(f"partID={partID}").show(10, False)

In [ ]:
tri_par_titre(film_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  23|
|     1|  25|
|     2|  23|
|     3|  21|
+------+----+

partition 0
+---+-------------------------------------+--------------------------------+----------+------+
|nF |titre                                |genres                          |HushNumber|partID|
+---+-------------------------------------+--------------------------------+----------+------+
|19 |Ace Ventura: When Nature Calls (1995)|[Comedy]                        |0         |0     |
|37 |Across the Sea of Time (1995)        |[Documentary, IMAX]             |0         |0     |
|11 |American President, The (1995)       |[Comedy, Drama, Romance]        |0         |0     |
|85 |Angels and Insects (1995)            |[Drama, Romance]                |0         |0     |
|82 |Antonia's Line (Antonia) (1995)      |[Comedy, Drama]                 |0         |0     |
|23 |Assassins (1995)                     |[Action, Crime, Thriller]       |0         |0     |
|34 |Babe (1995)    

**Comments (using repartition command)**

* The repartitioning based on the HushNumber ensures that the rows having the same number (aka. the same first letter) are regrouped in one partition (together).
* **However**, The repartitioning does not take into account the order of the numbers. There is no insurance that 1 and 2 are in the same/adjacent partitions.
  * This is due to the inner functionning of the command 'repartition'



## Question 2 : Regrouper les films par genre et les compter

En vous inspirant de la fonction *topN*  définir une fonction qui compte le nombre de films par genre. Le schéma du résultat est (genre, n)

In [ ]:
def count_par_genre(df):
  def countPartGenres(partID, iterateur):
    suivant = next(iterateur, None)
    genres=dict()
    while suivant is not None :
      for g in suivant.genres: #a list
        if g in genres:
          genres[g] +=1
        else:
          genres[g]= 1
      suivant = next(iterateur, None)
    
    for genre, nb in genres.items():
        yield (partID, genre, nb)

  def countPartGenre(partID, iterateur):
    suivant = next(iterateur, None)
    genres=dict()
    while suivant is not None :
      g = suivant.genre
      if g in genres:
        genres[g] += suivant.n
      else:
        genres[g]= suivant.n
      suivant = next(iterateur, None)
    
    for genre, nb in genres.items():
        yield (partID, genre, nb)
    
  nom_attributs = ['partID','genre','n']

  hashed_df = df.rdd.mapPartitionsWithIndex(countPartGenres)
  .toDF(nom_attributs).repartitionByRange(3, col("genre"))
  .rdd.mapPartitionsWithIndex(countPartGenre).toDF(nom_attributs)
  #does map not regroup data  = No

  # hashed_df = hashed_df.groupBy(col("genre"))
  #.agg(sum(col("n")).alias("n"))
  showPartitionSize(hashed_df)
  return hashed_df
      

In [ ]:
res = count_par_genre(film_extrait)
display(res)

+------+----+
|partID|size|
+------+----+
|     0|   6|
|     1|   6|
|     2|   5|
+------+----+



,partID,genre,n
0,0,Comedy,31
1,0,Children,12
2,0,Action,17
3,0,Crime,14
4,0,Adventure,12
5,0,Animation,3
6,1,Drama,50
7,1,Horror,5
8,1,Fantasy,5
9,1,Documentary,3


## Question 3 : Numérotation

En vous inspirant de la fonction *topN*  définir une fonction qui attribue un numéro de 1 à n à chaque tuple de notes_extrait. Le résultat a les attributs de notes + un attribut num.

In [ ]:
def numeroter(df):
  nb_partitions = df.rdd.getNumPartitions()

  stats = showPartitionSize(df).collect() 
  #with collect: it returns a list of rows.
  stats = dict(stats) #return partID: number

  #track the current numbers across partition
  partIDs = list(stats.keys())
  partIDs.sort()
  #from first to last partition (we can not sort them 
  #(in this case, the order of numbers may not be preserved.))
  sum = 1
  res = dict()
  for i in partIDs:
    res[i]=sum
    sum += stats[i]

  broadcastRes = spark.sparkContext.broadcast(res)

  def numPart(partID, iterateur):
    num = broadcastRes.value[partID] #numInterne
    suivant = next(iterateur, None)
    while suivant is not None :
      yield (partID, *suivant, num)
      num+=1
      suivant = next(iterateur, None)

  nom_attributs = ['partID'] + df.schema.fieldNames() + ['num']

  numbered_df = df.rdd.mapPartitionsWithIndex(numPart)
  .toDF(nom_attributs)

  # showPartitions(numbered_df)

  # afficher séparément le contenu de chaque partition
  for partID in range(nb_partitions):
    print("partition", partID)
    numbered_df.where(f"partID={partID}").show(10, False)

numeroter(notes_extrait)

+------+----+
|partID|size|
+------+----+
|     0|  62|
|     1|  62|
|     2|  61|
+------+----+

partition 0
+------+---+---+----+-----+---+
|partID|nF |nU |note|annee|num|
+------+---+---+----+-----+---+
|0     |19 |76 |1.5 |2011 |1  |
|0     |62 |691|5.0 |1996 |2  |
|0     |25 |832|5.0 |1998 |3  |
|0     |5  |296|3.5 |2005 |4  |
|0     |48 |781|3.0 |1996 |5  |
|0     |34 |871|4.0 |2000 |6  |
|0     |2  |796|1.5 |2015 |7  |
|0     |2  |353|3.0 |2003 |8  |
|0     |28 |815|5.0 |1996 |9  |
|0     |5  |196|4.0 |1996 |10 |
+------+---+---+----+-----+---+
only showing top 10 rows

partition 1
+------+---+---+----+-----+---+
|partID|nF |nU |note|annee|num|
+------+---+---+----+-----+---+
|1     |15 |875|4.0 |1996 |63 |
|1     |62 |368|4.0 |2006 |64 |
|1     |29 |592|4.0 |2000 |65 |
|1     |31 |767|3.0 |1996 |66 |
|1     |25 |339|5.0 |1997 |67 |
|1     |34 |682|4.0 |2000 |68 |
|1     |19 |63 |0.5 |2007 |69 |
|1     |19 |240|1.5 |2003 |70 |
|1     |2  |390|2.0 |2005 |71 |
|1     |19 |533|3.0

## Question 4 : Regrouper et trier

Définir une fonction qui affiche le nombre de notes par année dans l'ordre croissant des années

In [ ]:
def regrouper_trier(df):
  def countPartYear(partID, iterateur):
    suivant = next(iterateur, None)
    years=dict()
    while suivant is not None :
      if suivant.annee in years:
        years[suivant.annee] +=1
      else:
        years[suivant.annee] =1
      suivant = next(iterateur, None)
    
    for year, nb in years.items():
        yield (partID, year, nb)

  def countTotalYear(partID, iterateur):
    suivant = next(iterateur, None)
    years=dict()
    while suivant is not None :
      if suivant.year in years:
        years[suivant.year] += suivant.n
      else:
        years[suivant.year] = suivant.n
      suivant = next(iterateur, None)
    
    for year, nb in years.items():
        yield (partID, year, nb)
    
  nom_attributs = ['partID','year','n']

  year_df_part = df.rdd.mapPartitionsWithIndex(countPartYear)
  .toDF(nom_attributs).repartitionByRange(3, col("year"))
  .sortWithinPartitions(col("year"))
  .rdd.mapPartitionsWithIndex(countTotalYear)
  .toDF(nom_attributs)
  final_df = year_df_part
  # final_df = year_df_part.groupBy(col("year"))
  #.agg(sum(col("n")).alias("n")).orderBy(col("year"))
  #.selectExpr("year", "n")
  final_df.show()
regrouper_trier(notes_extrait)

+------+----+---+
|partID|year|  n|
+------+----+---+
|     0|1996| 52|
|     0|1997| 20|
|     0|1998|  3|
|     0|1999| 12|
|     0|2000| 14|
|     0|2001| 11|
|     0|2002|  6|
|     1|2003|  9|
|     1|2004|  7|
|     1|2005|  4|
|     1|2006|  9|
|     1|2007|  7|
|     1|2008|  1|
|     1|2009|  5|
|     2|2010|  3|
|     2|2011|  1|
|     2|2012|  2|
|     2|2013|  1|
|     2|2014|  3|
|     2|2015|  5|
+------+----+---+
only showing top 20 rows



In [ ]:
notes_extrait.groupBy(col("annee")).agg(count("*")).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[annee#92L], functions=[count(1)])
   +- Exchange hashpartitioning(annee#92L, 4), ENSURE_REQUIREMENTS, [plan_id=1861]
      +- HashAggregate(keys=[annee#92L], functions=[partial_count(1)])
         +- InMemoryTableScan [annee#92L]
               +- InMemoryRelation [nF#89L, nU#90L, note#91, annee#92L], StorageLevel(disk, memory, deserialized, 1 replicas)
                     +- Exchange RoundRobinPartitioning(3), REPARTITION_BY_NUM, [plan_id=85]
                        +- *(5) Project [nF#89L, nU#90L, note#91, annee#92L]
                           +- *(5) SortMergeJoin [nF#89L], [nF#0L], Inner
                              :- *(2) Sort [nF#89L ASC NULLS FIRST], false, 0
                              :  +- Exchange hashpartitioning(nF#89L, 4), ENSURE_REQUIREMENTS, [plan_id=69]
                              :     +- *(1) Filter ((isnotnull(nU#90L) AND (nU#90L < 1000)) AND isnotnull(nF#89L))
                    

**IN THE EXPLAIN() RESULT, FOCUS ON THE FIRST LEVELS:**
- exchange
- hash aggregate
- join
- sort

## Question 5 : top fréquence

Définir une fonction qui affiche les 5 mots les plus fréquents dans les titres

In [ ]:
import re
import string
st = "My name is Lina (2002)"
res = re.sub('['+string.punctuation+']', '', st)
.split()[:-1]
res

['My', 'name', 'is', 'Lina']

In [ ]:
def top_frequence(df):
  def countPartWords(partID, iterateur):
    suivant = next(iterateur, None)
    words=dict()
    while suivant is not None :
      ws = re.sub('['+string.punctuation+']', '', suivant.titre)
      .lower().split()[:-1]#we ignore punctuation + the production year.
      for w in ws:
        if w in words:
          words[w] +=1
        else:
          words[w] =1
      suivant = next(iterateur, None)
    
    for w, nb in words.items():
        yield (partID, w, nb)

  def countPartWord(partID, iterateur):
    suivant = next(iterateur, None)
    words=dict()
    while suivant is not None :
      w = suivant.word
      if w in words:
        words[w] += suivant.n
      else:
        words[w] = suivant.n
      suivant = next(iterateur, None)
    
    for w, nb in words.items():
        yield (w, nb)
    
  nom_attributs = ['partID','word','n']
  nom_attributs_final = ['word','n']

  year_df_part = df.rdd.mapPartitionsWithIndex(countPartWords)
  .toDF(nom_attributs).repartitionByRange(3, col("word"))
  .rdd.mapPartitionsWithIndex(countPartWord)
  .toDF(nom_attributs_final)
  final_df = year_df_part
  .repartitionByRange(3, col("n"))
  .sortWithinPartitions("n").limit(20)
  # final_df = year_df_part.groupBy(col("word"))
  #.agg(sum(col("n")).alias("n"))
  #.orderBy(col("n").de sc()).limit(20)
  final_df.show()
top_frequence(film_extrait)

+--------+---+
|    word|  n|
+--------+---+
|     the| 18|
|      in|  7|
|     and|  6|
|      to|  5|
|      of|  5|
|    dead|  4|
|    when|  4|
|       a|  3|
|     for|  3|
|    time|  3|
|     yao|  3|
|      an|  2|
|american|  2|
|     eye|  2|
|     aka|  2|
|      it|  2|
|      la|  2|
|     man|  2|
| monkeys|  2|
|   white|  2|
+--------+---+



## Question 6 : Jointure

Définir les fonctions pour calculer la jointure entre les extraits de films et de notes

In [ ]:
showPartitionSize(film_extrait)
showPartitionSize(notes_extrait)
#Comment: Notes extrait is double the size of the films df 

+------+----+
|partID|size|
+------+----+
|     0|  31|
|     1|  31|
|     2|  30|
+------+----+

+------+----+
|partID|size|
+------+----+
|     0|  62|
|     1|  62|
|     2|  61|
+------+----+



DataFrame[partID: bigint, size: bigint]

In [ ]:
def jointure_film_notes(df1, df2):
  k = 4 # target partitions

  def hashFun(num, k=2):
    return (num%k)

  def addHashNumber(partID, iterateur):
    suivant = next(iterateur, None)
    while suivant is not None :
      yield (*suivant, hashFun(suivant["nF"],k))
      suivant = next(iterateur, None)

  def mergeRDDs(partID, iterateur):
    suivant = next(iterateur, None)
    while suivant is not None :
      yield (*suivant, hashFun(suivant["nF"],k))
      suivant = next(iterateur, None)

  nom_attributs_1 = df1.schema.fieldNames() + ['hashPartition']
  table1_df = df1.rdd.mapPartitionsWithIndex(addHashNumber)
  .toDF(nom_attributs_1).repartitionByRange(k, col("hashPartition"))
  .persist()
  table1_df_sorted = table1_df.sortWithinPartitions(col("nF")) 
  #nF =clé de jointure

  nom_attributs_2 = df2.schema.fieldNames() + ['hashPartition']
  table2_df = df2.rdd.mapPartitionsWithIndex(addHashNumber)
  .toDF(nom_attributs_2).repartitionByRange(k, col("hashPartition"))
  .persist()
  table2_df_sorted = table2_df.sortWithinPartitions(col("nF"))

  #nF are both in the first position
  
  # inter  = table2_df_sorted.rdd.cogroup(table1_df_sorted.rdd).toDF()
  # print(inter)
  # inter.show()

  #TODO: MAKE THE REPARTITION ASSIGN SAME HUSH NUMBERS 
  #TO SAME PARTITIONS BETWEEN THE 2 TABLES ==> Using repartition By Range 
  res = table2_df_sorted.join(table1_df_sorted, 
  table2_df_sorted.hashPartition == table1_df_sorted.hashPartition)
  # res.explain()
  showPartitions(res)

  # showPartitionSize(table1_df)

  # hashed_df = hashed_df.withColumn("partID", 
  #spark_partition_id()).sortWithinPartitions(col("titre"))

  # nb_partitions = hashed_df.rdd.getNumPartitions()

  # afficher séparément le contenu de chaque partition
  # for partID in range(nb_partitions):
  #   print("partition", partID)
  #   # afficher la partition partID
  #   hashed_df.where(f"partID={partID}").show(10, False)
jointure_film_notes(film_extrait, notes_extrait)

+------+---+---+----+-----+-------------+---+-------------------------------------------+----------------------------------------+-------------+
|partID|nF |nU |note|annee|hashPartition|nF |titre                                      |genres                                  |hashPartition|
+------+---+---+----+-----+-------------+---+-------------------------------------------+----------------------------------------+-------------+
|0     |48 |781|3.0 |1996 |0            |80 |White Balloon, The (Badkonake sefid) (1995)|[Children, Drama]                       |0            |
|0     |48 |781|3.0 |1996 |0            |88 |Black Sheep (1996)                         |[Comedy]                                |0            |
|0     |48 |781|3.0 |1996 |0            |20 |Money Train (1995)                         |[Action, Comedy, Crime, Drama, Thriller]|0            |
|0     |48 |781|3.0 |1996 |0            |12 |Dracula: Dead and Loving It (1995)         |[Comedy, Horror]                        |

In [ ]:
#END